In [ ]:
from tensorflow.keras import backend as K

from keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Lambda,Conv1D,Conv2DTranspose, LeakyReLU,Activation,Flatten,Reshape
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import load_model
import emcee
import warnings
warnings.filterwarnings('ignore')

In [ ]:
JV_raw = np.loadtxt("/mnt/c/Users/pssrg/Downloads/GaAs_sim_nJV.txt")
par = np.loadtxt("/mnt/c/Users/pssrg/Downloads/GaAs_sim_label.txt")


In [ ]:
def Conv1DTranspose(input_tensor, filters, kernel_size, strides ):
    
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
    x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1),padding='SAME')(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    return x

#Covert labels from log10 form to log
        
def log10_ln(x):
    return np.log(np.power(10,x))

par = log10_ln(par)


#Data normalization for the whole JV dataset

def min_max(x):
    min = np.min(x)
    max = np.max(x)
    return (x-min)/(max-min),max,min

#Normalize raw JV data

JV_norm,JV_max,JV_min = min_max(JV_raw)

#Normalize JV descriptors column-wise
scaler = MinMaxScaler()

par_n = scaler.fit_transform(par)   

#create training and testing datset

X_train, X_test, y_train, y_test = train_test_split(JV_norm,par_n, test_size=0.2)

#add in Gaussian noise to train the denoising Autoencoder

X_train_nos = X_train+0.002 * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape) 

X_test_nos = X_test+0.002 * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K

def build_regression_model(label_dim=5, max_filter=256, map_size = 25, kernel=[7, 5, 3], strides=[5, 2, 2]):
    z_in = tf.keras.Input(shape=(label_dim,))  # input: material descriptors (5,)
    
    # Dense expansion
    z = layers.Dense(100, activation='relu')(z_in)
    z = layers.Dense(max_filter * map_size, activation='relu')(z)
    z = layers.Reshape((map_size, 1, max_filter))(z)
    
    # Deconvolution blocks
    z = layers.Conv2DTranspose(max_filter // 2, (kernel[2], 1), strides=(strides[2], 1), padding='same')(z)
    z = layers.Activation('relu')(z)
    
    z = layers.Conv2DTranspose(max_filter // 4, (kernel[1], 1), strides=(strides[1], 1), padding='same')(z)
    z = layers.Activation('relu')(z)
    
    z = layers.Conv2DTranspose(1, (kernel[0], 1), strides=(strides[0], 1), padding='same')(z)
    z = layers.Activation('sigmoid')(z)

    # Remove singleton dimensions (squeeze)
    z = layers.Lambda(lambda x: tf.squeeze(x, axis=2))(z)  # remove width = 1
    z = layers.Lambda(lambda x: tf.squeeze(x, axis=2))(z)  # remove channel = 1

    model = tf.keras.Model(inputs=z_in, outputs=z)
    return model

In [ ]:
reg_model = build_regression_model()
reg_model.compile(optimizer='adam', loss='mse')

reg_model.fit(par_n, JV_norm, batch_size=128, epochs=50, shuffle=True)

In [ ]:
y_hat_train = reg_model(y_train, training = False)
y_hat_test = reg_model(y_test, training = False)


y_hat_train_np = y_hat_train.numpy()
y_hat_test_np = y_hat_test.numpy()
X_train_np = X_train.numpy() if isinstance(X_train, tf.Tensor) else X_train
X_test_np = X_test.numpy() if isinstance(X_test, tf.Tensor) else X_test


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import time
import os

tfd = tfp.distributions
tfb = tfp.bijectors

# ------------- Inputs -------------
T_vals = np.array([530, 580, 630, 650, 680], dtype=np.float32)
x_vals = -1000.0 / T_vals
JV_exp = tf.convert_to_tensor(np.loadtxt('/mnt/c/Users/pssrg/Downloads/GaAs_exp_nJV.txt'), dtype=tf.float32)

# ------------- Log-Probability Function -------------
@tf.function(reduce_retracing=True)
def log_prob(*theta):
    x = tf.convert_to_tensor(x_vals, dtype=tf.float32)
    inv_x = -1.0 / x
    log_input = tf.clip_by_value(inv_x, 1e-6, 1e6)

    try:
        if tf.reduce_any([tf.math.is_nan(t) for t in theta]):
            tf.print("💀 NaN detected in θ input → skipping this point")
            return tf.constant(-1e6, dtype=tf.float32)

        a1, b1, c1, a2, b2, c2, a3, b3, c3, a4, b4, c4, a5, b5, c5 = theta
        params = tf.stack(theta)
        bounds_low = tf.constant([-5.0, -10.0, -10.0] * 5, dtype=tf.float32)
        bounds_high = tf.constant([5.0, 10.0, 10.0] * 5, dtype=tf.float32)

        if tf.reduce_any(tf.logical_or(params < bounds_low, params > bounds_high)):
            return tf.constant(-1e6, dtype=tf.float32)

        def descriptor_fn(a, b, c):
            return a * tf.math.log(log_input) + b * x + c

        descriptor_scaled = 10 * tf.stack([
            descriptor_fn(a1, b1, c1),
            descriptor_fn(a2, b2, c2),
            descriptor_fn(a3, b3, c3),
            descriptor_fn(a4, b4, c4),
            descriptor_fn(a5, b5, c5),
        ], axis=-1)

        jv_pred = reg_model(descriptor_scaled, training=False)
        sigma = tf.constant(1e-4, dtype=tf.float32)
        loss = tf.reduce_mean(tf.square((JV_exp - jv_pred) / sigma))
        log_likelihood = -0.5 * loss
#        tf.print("📈 log_likelihood:", log_likelihood)

        return tf.clip_by_value(log_likelihood, -1e6, 0.0)

    except Exception as e:
        tf.print("⚠️ Exception in log_prob():", e)
        return tf.constant(-1e6, dtype=tf.float32)


# ------------- Sampling Setup -------------

num_results = 100
num_burnin_steps = 50
num_params = 15

filename = 'mcmc_samples.dat'
if os.path.exists(filename):
    os.remove(filename)

mmap_samples = np.memmap(filename, dtype='float32', mode='w+', shape=(num_results, num_params))

initial_theta = 0.0 + 0.0001 * np.random.randn(num_params)
initial_state = [tf.constant(v, dtype=tf.float32) for v in initial_theta]

nuts_kernel = tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=log_prob,
    step_size=0.0001
)

adaptive_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
    inner_kernel=nuts_kernel,
    num_adaptation_steps=int(0.8 * num_burnin_steps),
    target_accept_prob=0.8
)

# ------------- Run Sampling -------------
def run_nuts():
    t0 = time.time()

    samples, log_probs = tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_state,
        kernel=adaptive_kernel,
        trace_fn=lambda cs, kr: kr.inner_results.target_log_prob
    )

    samples_np = tf.stack(samples, axis=1).numpy()  # shape: (num_results, 15)
    mmap_samples[:] = samples_np
    mmap_samples.flush()

#    print(f"✅ Sampling complete. Time: {time.time() - t0:.2f} seconds")
 #   print("📁 Samples saved to:", filename)
  #  print("📉 Mean log_prob:", tf.reduce_mean(log_probs).numpy())
    return samples_np, log_probs

samples, log_probs = run_nuts()

In [ ]:
theta_last = samples_2[-1, :]  # Shape: (15,)
sim_JVs, _ = check_plot(theta_last, x_vals, 1)  # x is your experimental temperature grid

# ---- Plot simulated vs experimental JV curves ----

fig, ax = plt.subplots(5, 1, figsize=(8, 10))
for i in range(5):
    ax[i].plot(sim_JVs[i, :], '--', label='Simulated')
    ax[i].plot(JV_exp[i, :], label='Experimental')
    ax[i].set_ylabel(f'Curve {i+1}')
    ax[i].legend()
ax[-1].set_xlabel("Voltage Index (or Time)")
plt.suptitle("Simulated vs Experimental JV Curves")
plt.tight_layout()
plt.show()

# ---- Extract material properties over a finer x grid ----

x_step = np.linspace(min(x), max(x), 50)

par_in = []
for i in range(samples_2.shape[0]):
    try:
        _, par_input = check_plot(samples_2[i, :], x_vals, 0)
        if par_input.shape == (len(x_vals), 5):
            par_in.append(par_input)
    except Exception as e:
        print(f"Skipping sample {i}: {e}")

par_in = np.array(par_in)  # Shape: (n_samples, 50, 5)
print(par_in.shape)

# ---- Discard burn-in (optional) ----
if par_in.shape[0] > 2000:
    par_in = par_in[2000:, :, :]

# ---- Convert from log-space to actual values ----
par_in = np.exp(par_in)  # Final shape: (n_good_samples, 50, 5)





In [ ]:
def plot_uncertain(x,y):
    
    mu = np.mean(y,axis = 0)
    std = np.std(y, axis = 0)
    plt.fill_between(x, mu+std,mu-std,alpha=0.1,color='grey')
    plt.plot(x,mu,color='black')

plt.rcParams["figure.figsize"] = [8, 10]
plt.rcParams.update({'font.size': 16})
 
fig = plt.figure()
y_label = ['Conc.[cm-3]','Conc.[cm-3]', r'$\tau$ [s]', 'SRV [cm/S]','SRV [cm/S]']
x_labels = ['-1/530' ,'-1/580','-1/630','-1/680']
title = ['Zn emitter doping' , 'Si base doping' ,'bulk lifetime','Front SRV', 'Rear SRV']


for i in range(5):
    plt.subplot(5,1,i+1)
    
    l1=plot_uncertain(x_vals,par_in[:,:,i]) 
   
    plt.yscale('log') 
    plt.ylabel(y_label[i])
    plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680],[])
    plt.title(title[i],fontsize=15,fontweight='bold')
    plt.xlim(-1000/530,-1000/680)
    
  
plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680], x_labels)

plt.xlabel(r'-1/T [1/C]') 

fig.align_labels()

In [ ]:
idx_map = np.argmax(log_probs)
theta_last = samples[idx_map, :]  # Shape: (15,)
sim_JVs, _ = check_plot(theta_last, x_vals, 1)  # x is your experimental temperature grid

# ---- Plot simulated vs experimental JV curves ----

fig, ax = plt.subplots(5, 1, figsize=(8, 10))
for i in range(5):
    ax[i].plot(sim_JVs[i, :], '--', label='Simulated')
    ax[i].plot(JV_exp[i, :], label='Experimental')
    ax[i].set_ylabel(f'Curve {i+1}')
    ax[i].legend()
ax[-1].set_xlabel("Voltage Index (or Time)")
plt.suptitle("Simulated vs Experimental JV Curves")
plt.tight_layout()
plt.show()

In [ ]:

_, par_input = check_plot(samples[idx_map,:], x_vals, 0)
par_in = np.exp(par_input)

def plot_uncertain_2(x,y):
    plt.plot(x,y,color='black')

plt.rcParams["figure.figsize"] = [8, 10]
plt.rcParams.update({'font.size': 16})
 
fig = plt.figure()
y_label = ['Conc.[cm-3]','Conc.[cm-3]', r'$\tau$ [s]', 'SRV [cm/S]','SRV [cm/S]']
x_labels = ['-1/530' ,'-1/580','-1/630','-1/680']
title = ['Zn emitter doping' , 'Si base doping' ,'bulk lifetime','Front SRV', 'Rear SRV']

for i in range(5):
    plt.subplot(5,1,i+1)
    
    l1=plot_uncertain_2(x_vals,par_in[:,i]) 
    plt.yscale('log') 
    plt.ylabel(y_label[i])
    plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680],[])
    plt.title(title[i],fontsize=15,fontweight='bold')
    plt.xlim(-1000/530,-1000/680)
    
  
plt.xticks([-1000/530,-1000/580,-1000/630,-1000/680], x_labels)

plt.xlabel(r'-1/T [1/C]') 

fig.align_labels()